# --- Environment Setup ---

In [1]:
!pip install -q gdown

import os
import json
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models
from PIL import Image
from sklearn.model_selection import train_test_split

# --- Download and Extract Data ---

In [2]:
import gdown
url = 'https://drive.google.com/uc?id=1ptdvy-EAT4IFtnvD1ZGDgRRhEigN1hyU'
output = 'fathomnet.zip'
gdown.download(url, output, quiet=False)
!unzip -q fathomnet.zip -d /kaggle/working/fathomnet_unzipped

Downloading...
From (original): https://drive.google.com/uc?id=1ptdvy-EAT4IFtnvD1ZGDgRRhEigN1hyU
From (redirected): https://drive.google.com/uc?id=1ptdvy-EAT4IFtnvD1ZGDgRRhEigN1hyU&confirm=t&uuid=bbe3fd17-b4ce-4ec4-8b37-e448b3302054
To: /kaggle/working/fathomnet.zip
100%|██████████| 1.25G/1.25G [00:11<00:00, 105MB/s] 


In [3]:
import gdown
url = 'https://drive.google.com/uc?id=1eOtHFtawo1pmNzAomdvWEZkpZFoE-4xU'
output = 'full_taxonomy.json'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1eOtHFtawo1pmNzAomdvWEZkpZFoE-4xU
To: /kaggle/working/full_taxonomy.json
100%|██████████| 19.1k/19.1k [00:00<00:00, 19.8MB/s]


'full_taxonomy.json'

# --- Paths ---

In [4]:
DATA_DIR = "/kaggle/working/fathomnet_unzipped"
TRAIN_IMG_DIR = os.path.join(DATA_DIR, "train_data/rois")
TEST_IMG_DIR = os.path.join(DATA_DIR, "test_data/rois")
train_df = pd.read_csv(os.path.join(DATA_DIR, "train_data/annotations.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIR, "test_data/annotations.csv"))

print("Train:", len(train_df), "Test:", len(test_df))

Train: 23699 Test: 788


# --- Load Taxonomy File ---

In [5]:
with open("/kaggle/working/full_taxonomy.json") as f:
    taxonomy = json.load(f)

# Build label to best taxon and taxonomic depth dictionary
taxonomic_ranks = ["kingdom", "phylum", "class", "order", "family", "genus", "species"]
label_to_best_taxon = {}
label_to_depth_map = {}

for entry in taxonomy:
    for i, rank in enumerate(taxonomic_ranks):
        if entry[rank] != "unknown":
            label_to_best_taxon[entry["label"]] = entry[rank]
            label_to_depth_map[entry[rank]] = i
            break

# --- Apply Taxonomy to Train Labels ---

In [6]:
train_df['label'] = train_df['label'].map(label_to_best_taxon)
train_df['path'] = train_df['path'].apply(lambda x: os.path.basename(x))
test_df['path'] = test_df['path'].apply(lambda x: os.path.basename(x))

In [7]:
test_df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,path,label
0,1_1.png,NaN
1,2_2.png,NaN
2,2_3.png,NaN
3,2_4.png,NaN
4,2_5.png,NaN


# --- Dataset Class ---

# --- Transforms ---

In [8]:
class OceanDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.label_to_idx = {label: i for i, label in enumerate(sorted(self.df['label'].unique()))}
        self.idx_to_label = {i: label for label, i in self.label_to_idx.items()}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]['path'])
        image = Image.open(img_path).convert("RGB")
        label = self.label_to_idx[self.df.iloc[idx]['label']]
        if self.transform:
            image = self.transform(image)
        return image, label

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- Split Train/Validation ---

In [10]:
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

train_dataset = OceanDataset(train_df, TRAIN_IMG_DIR, transform)
val_dataset = OceanDataset(val_df, TRAIN_IMG_DIR, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# --- Model ---

In [11]:
num_classes = len(train_dataset.label_to_idx)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.cuda()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 181MB/s]


# --- Taxonomic Distance Matrix ---

In [12]:
distance_matrix = np.zeros((num_classes, num_classes), dtype=np.float32)
labels = list(train_dataset.label_to_idx.keys())

for i, label_i in enumerate(labels):
    for j, label_j in enumerate(labels):
        depth_i = label_to_depth_map.get(label_i, 0)
        depth_j = label_to_depth_map.get(label_j, 0)
        distance_matrix[i, j] = abs(depth_i - depth_j) if label_i != label_j else 0.0


# --- Custom Hierarchical Loss ---

In [13]:
class TaxonomicLoss(nn.Module):
    def __init__(self, distance_matrix):
        super().__init__()
        self.distance_matrix = torch.tensor(distance_matrix).cuda()

    def forward(self, preds, targets):
        probs = torch.softmax(preds, dim=1)
        batch_loss = 0.0
        for i in range(preds.size(0)):
            dists = self.distance_matrix[targets[i]]  # Distance to true class
            loss = torch.dot(probs[i], dists)
            batch_loss += loss
        return batch_loss / preds.size(0)

criterion = TaxonomicLoss(distance_matrix)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# --- Training Setup ---

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# --- Training Loop ---

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss, train_correct = 0.0, 0
        for images, labels in train_loader:
            images, labels = images.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels)

        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.cuda(), labels.cuda()
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels)

        print(f"Epoch {epoch+1}: Train Loss: {train_loss/len(train_loader.dataset):.4f}, "
              f"Train Acc: {train_correct/len(train_loader.dataset):.4f}, "
              f"Val Loss: {val_loss/len(val_loader.dataset):.4f}, "
              f"Val Acc: {val_correct/len(val_loader.dataset):.4f}")

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=15)


Epoch 1: Train Loss: 0.0000, Train Acc: 1.0000, Val Loss: 0.0000, Val Acc: 1.0000


# --- Inference on Test Set ---

In [ ]:
test_dataset = OceanDataset(test_df, TEST_IMG_DIR, transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

model.eval()
preds = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        preds.extend([test_dataset.idx_to_label[i] for i in predicted.cpu().numpy()])

# --submission--

In [ ]:
submission = pd.DataFrame({
    'annotation_id': test_df['annotation_id'],
    'concept_name': preds
})
submission.to_csv("submission.csv", index=False)